<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

### In this notebook, we will learn how to download individual units for a given `parcl id` and select the events of interest for those units using the Parcl Labs API.


#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

You will need a paid account to get your API, you can get it [here](https://dashboard.parcllabs.com/). 

Run in Colab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/getting_started/property_data_download.ipynb)

In [ ]:
%pip install --upgrade parcllabs

After installing the required libraries, we need to load them and start the Parcl Labs client with our `API_KEY`. The client is a Python library designed to facilitate and optimize the user experience with the Parcl Labs API. It handles searching, retrieving, and formatting the data for us, making it much easier to use our data by abstracting away complexity and allowing you to focus on building. As a reminder, while you can simply enter your `API_KEY`, it is recommended that you save it as an environment variable for added security. If you are using Colab, you can follow these [steps](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75).


In [ ]:
import os
from datetime import datetime, timedelta
import pandas as pd
from parcllabs import ParclLabsClient
import os
from datetime import datetime
try:
    from google.colab import drive, files  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False 

In [ ]:
client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"),
    limit=10
)

download_dir = '/content/downloads' if 'IN_COLAB' in globals() and IN_COLAB else 'downloads'
os.makedirs(download_dir, exist_ok=True)

print("Setup complete. You can now use the 'client' object to interact with the Parcl Labs API.")

With the client object, you can now interact with the Parcl Labs API. The client object has multiple methods that you can use to download data. You can find more information about the methods for the Parcl Labs Client [here](https://github.com/ParclLabs/parcllabs-python). In this case, we are interested in the [property search v2 endpoint](https://docs.parcllabs.com/reference/property_search_v2_v2_property_search_post-1), and for this example, we are getting all sales between $300,000 and $500,000 for single-family homes in the city of Pittsburgh, PA (`parcl_id`: `5377717`) that are between 2,000 and 4,000 square feet. In the example below, we define the necessary parameters to search for this information and then call the Parcl Labs Client to download the data for us. You will notice that we have a handful of additional parameters commented out; those help us narrow down our search, but for now, let's use the `parcl_ids`, `event_names`, `min_price`, and `max_price`, `property_types`, `min_sqft`, and `max_sqft`. 

We set the `include_property_details` to True, so every event record will also return property metadata. Additionally `include_events` is set to True while `include_full_event_history` is set to false so this query will return the event metadata for the events we queried (sales between $300,000 and $500,000 for single-family homes in the city of Pittsburgh) but not the entire event history for the resulting properties. The parameters defined in `search_params` are a critical tool on how to search for relevant information on the API. They are designed to provide our users with the necessary control to make their search as wide or as narrow as they wish.

If you are interested in other markets you can search them [following these steps](https://github.com/ParclLabs/parcllabs-cookbook/blob/main/examples/getting_started/search.ipynb).

In [ ]:
# Drop-in replacement: runs on Colab or locally; saves to download_dir
if 'download_dir' not in globals():
    download_dir = '/content/downloads' if 'IN_COLAB' in globals() and IN_COLAB else 'downloads'
    os.makedirs(download_dir, exist_ok=True)

# For a full list of available parameters along with their documentation, see the property search v2 endpoint documentation linked in the cell above.
search_params = {
    'parcl_ids': [5377717],  # One of Parcl ID, parcl property ids, or geo coordinates is required
    # 'parcl_property_ids': [78353317, 135921544],
    # 'geo_coordinates': {"latitude": 36.159445, "longitude": -86.483244, "radius": 1},
    'event_names': ["ALL_SOLD"],  # See docs for full list of event names
    # 'min_event_date': "2023-01-01",
    # 'max_event_date': "2024-12-31",
    'min_price': 300000,
    'max_price': 500000,
    # 'is_new_construction': False,
    # 'min_record_updated_date': "2024-01-01",
    # 'max_record_updated_date': "2024-12-31",
    # 'is_current_owner': True,
    # 'owner_name': ["BLACKSTONE"],
    # 'is_investor_owned': True,
    # 'is_owner_occupied': False,
    'include_property_details': True,
    'include_events': True,
    'include_full_event_history': False,
    'property_types': ["SINGLE_FAMILY"],
    # 'min_beds': 1,
    # 'max_beds': 5,
    # 'min_baths': 1,
    # 'max_baths': 3,
    'min_sqft': 2000,
    'max_sqft': 4000,
    'min_year_built': 2000,
    # 'max_year_built': 2020,
    # 'min_record_added_date': "2024-12-13",
    # 'max_record_added_date': "2024-12-31",
    # 'current_on_market_flag': True,
    # 'current_on_market_rental_flag': False,
    # 'limit': 100,
}

# Retrieve results
search_results, filter_data = client.property_v2.search.retrieve(**search_params)

print(f"Found {search_results['parcl_property_id'].nunique()} distinct properties and {len(search_results)} events matching the criteria.")
print(search_results.head(2))

# Save results
home_search_filename = f'pittsburgh_property_homes_{datetime.now().strftime("%Y-%m-%d")}.csv'
search_results_file_path = os.path.join(download_dir, home_search_filename)
search_results.to_csv(search_results_file_path, index=False)

print(f"Search results saved to {search_results_file_path}")

The newly added parameters have narrowed the results and helped us identify 179 relevant homes with 233 sale events in our desired market, having a targeted search helps our customers identify relevant properties. Note that your numbers may slightly differ as new data comes in daily for the event histories.

If we would like to access the entire event history for our resulting property universe, we can access this information by using the same [property search v2 endpoint](https://docs.parcllabs.com/reference/property_search_v2_v2_property_search_post-1) and query and simply changing the `include_full_event_history` parameter to True. Note that in order for the full event history to be returned, the `include_events` parameter must also be set to True, no event metadata will be returned if it is set to false.

In this example, we will use this method to retrieve the event history for the homes found in the search results.

In [ ]:
# For a full list of available parameters along with their documentation, see the property search v2 endpoint documentation linked in the cell above.
full_history_params = {
    'parcl_ids': [5377717],  # One of Parcl ID, parcl property ids, or geo coordinates is required
    'event_names': ["ALL_SOLD"], # See docs for full list of event names
    'min_price': 300000,
    'max_price': 500000,
    'include_property_details': True,
    'include_events': True,
    'include_full_event_history': True,
    'property_types': ["SINGLE_FAMILY"],
    'min_sqft': 2000,
    'max_sqft': 4000,
    'min_year_built': 2000,
}

# we can pass the full_history_params dictionary to the retrieve method to get the search results using **full_history_params
full_history_results, filter_data = client.property_v2.search.retrieve(**full_history_params)

print(f"Found {full_history_results['parcl_property_id'].nunique()} distinct properties and {len(full_history_results)} events matching the criteria.")
print(full_history_results.head(2))

The result we get is the same 179 property universe but now with 1079 events - the full event history of our property universe. When you are ready to save your data for you can use the following code to save the data to a CSV file.

In [ ]:
# Drop-in replacement: runs on Colab or locally; saves to download_dir
if 'download_dir' not in globals():
    download_dir = '/content/downloads' if 'IN_COLAB' in globals() and IN_COLAB else 'downloads'
    os.makedirs(download_dir, exist_ok=True)

# Save the event results to a CSV file using today's date in the filename for easier tracking
events_filename = f'pittsburgh_property_events_all_events_{datetime.now().strftime("%Y-%m-%d")}.csv'
events_file_path = os.path.join(download_dir, events_filename)
full_history_results.to_csv(events_file_path, index=False)

print(f"Event history saved to {events_file_path}")
print(f"Total events retrieved: {len(full_history_results)}")